# Real Estate WebSite Scrapping 

### 2. Data Cleaning & Integration
### •	Use Python functions to clean the data (missing values, inconsistent formatting, duplicates, etc.).
### •	Combine data from both websites into a single cleaned CSV file.
### •	Ensure consistency in column naming and data types.


### Cleanning process of both cvs files 

In [47]:
import pandas as pd
import numpy as np

# Load the CSV file into a DataFrame named tibiaan2
tb = pd.read_csv('tibiaan1.csv')
# Optional: Display the first few rows to confirm it loaded correctly
print(tb.head())

# Drop 'Property_ID' and 'URL' columns from tibiaan2 as its uncsessary
tb.drop(['Property_ID', 'URL'], axis=1, inplace=True)
print("Remaining columns after dropping:")
print(tb.columns)

# Check what columns we have
print("Data shape:", tb.shape)
print("Column names:", list(tb.columns))

# Clean price column - remove text and convert to numbers
if 'Price' in tb.columns:
    # Remove OMR and commas from price
    tb['Price'] = tb['Price'].astype(str).str.replace('OMR', '').str.replace(',', '').str.strip()
    # Convert to numbers
    tb['Price'] = pd.to_numeric(tb['Price'], errors='coerce')
    print("Price column cleaned")

# Clean location column
if 'Location' in tb.columns:
    # Fill empty locations with most common location
    most_common_location = tb['Location'].mode()[0]
    tb['Location'] = tb['Location'].fillna(most_common_location)
    print(f"Filled missing locations with: {most_common_location}")

# Clean bedrooms
if 'Bedrooms' in tb.columns:
    # Extract numbers from bedroom text
    tb['Bedrooms'] = tb['Bedrooms'].astype(str).str.extract(r'(\d+)')[0]
    tb['Bedrooms'] = pd.to_numeric(tb['Bedrooms'], errors='coerce')
    # Fill missing with average
    avg_bedrooms = tb['Bedrooms'].mean()
    tb['Bedrooms'] = tb['Bedrooms'].fillna(avg_bedrooms)
    tb['Bedrooms'] = tb['Bedrooms'].astype(int)
    print("Bedrooms column cleaned")

# Clean bathrooms same way
if 'Bathrooms' in tb.columns:
    tb['Bathrooms'] = tb['Bathrooms'].astype(str).str.extract(r'(\d+)')[0]
    tb['Bathrooms'] = pd.to_numeric(tb['Bathrooms'], errors='coerce')
    avg_bathrooms = tb['Bathrooms'].mean()
    tb['Bathrooms'] = tb['Bathrooms'].fillna(avg_bathrooms)
    tb['Bathrooms'] = tb['Bathrooms'].astype(int)
    print("Bathrooms column cleaned")

# Clean title column
if 'Title' in tb.columns:
    tb['Title'] = tb['Title'].fillna('No Title')
    
    # Make property type column from title
    def get_property_type(title):
        title = str(title).lower()
        if 'apartment' in title or 'flat' in title:
            return 'Apartment'
        elif 'villa' in title or 'house' in title:
            return 'Villa'
        elif 'office' in title:
            return 'Office'
        elif 'land' in title or 'plot' in title:
            return 'Land'
        else:
            return 'Other'
    
    tb['Property_Type'] = tb['Title'].apply(get_property_type)
    print("Property type column created")

# Remove last row if its mostly empty
last_row = tb.iloc[-1]
empty_count = last_row.isnull().sum()
total_cols = len(tb.columns)
if empty_count > total_cols / 2:
    tb = tb.iloc[:-1]
    print("Removed last row (mostly empty)")

# Check for missing values
print("\nMissing values after cleaning:")
print(tb.isnull().sum())

# Save cleaned data
tb.to_csv('cleaned_tibiaan.csv', index=False)
print("\nSaved cleaned data to cleaned_tibiaan.csv")
print("Final shape:", tb.shape)

# Show first few rows
print("\nFirst 3 rows of cleaned data:")
print(tb.head(3))

# Show some stats
print("\nData summary:")
if 'Price' in tb.columns:
    print(f"Price range: {tb['Price'].min():,.0f} to {tb['Price'].max():,.0f}")
if 'Property_Type' in tb.columns:
    print("Property types:")
    print(tb['Property_Type'].value_counts())

   Property_ID                                              Title  \
0         3116  Furnished Villa for Sale in Al Maabela – Resor...   
1         3096  Luxury furnished villa for sale with 8 bedroom...   
2         3094               Newly built Flat for Sale in Salalah   
3         3089                          Flat for sale in Al Qurum   
4         3088                          Land for slae in Al Qurum   

                                            Location        Price      Size  \
0  Furnished Villa for Sale in Al Maabela – Resor...   80,000 OMR   157 sqm   
1  Luxury furnished villa for sale with 8 bedroom...  415,000 OMR       NaN   
2                                                NaN   55,000 OMR   111 sqm   
3                          Flat for sale in Al Qurum   45,000 OMR       NaN   
4                          Land for slae in Al Qurum  525,000 OMR  1750 sqm   

   Bedrooms  Bathrooms                                                URL  
0       2.0        3.0  https://ti

In [48]:
tb

,Title,Location,Price,Size,Bedrooms,Bathrooms,Property_Type
0,Furnished Villa for Sale in Al Maabela – Resor...,Furnished Villa for Sale in Al Maabela – Resor...,80000,157 sqm,2,3,Villa
1,Luxury furnished villa for sale with 8 bedroom...,Luxury furnished villa for sale with 8 bedroom...,415000,NaN,8,11,Villa
2,Newly built Flat for Sale in Salalah,Bausher,55000,111 sqm,2,3,Apartment
3,Flat for sale in Al Qurum,Flat for sale in Al Qurum,45000,NaN,2,3,Apartment
4,Land for slae in Al Qurum,Land for slae in Al Qurum,525000,1750 sqm,5,5,Land
...,...,...,...,...,...,...,...
374,Apartment for Sale in Al Qurum,Apartment for Sale in Al Qurum,75000,95 sqm,3,3,Apartment
375,3 BHK Apartment for Sale in Al Amrat,3 BHK Apartment for Sale in Al Amrat,45000,110 sqm,5,5,Apartment
376,Land for Sale in Al Maabilah,Land for Sale in Al Maabilah,59000,600 sqm,5,5,Land
377,Twin Villa for Sale in Madinat As Sultan Qaboos,Bausher,230000,379 sqm,5,5,Villa


# Vista Oman Data 

In [49]:
# Load the CSV file into a DataFrame named vista2
Vista1 = pd.read_csv('V1.csv')

# Optional: Display the first few rows to confirm it loaded correctly
Vista1.head()

,title,price,bedrooms,bathrooms,city,property_type,email
0,1 BR + Maid’s Room Excellent Furnished Apartme...,OMR 50000,1.0,1.0,Bousher,Apartment,info@vistaoman.com
1,1 Desk Office Space for Small Companies in Qurum,OMR 20000,NaN,NaN,Qurum,Office,info@vistaoman.com
2,5 BR + 1 + 1 Rooms Luxury Golf-View Villa with...,OMR 230000,5.0,5.0,Muscat Hills,Villa,info@vistaoman.com
3,1 BR Cozy Freehold Penthouse Apartment – Musca...,OMR 65000,1.0,1.0,NaN,Penthouse,info@vistaoman.com
4,5 BR Spacious Villa in Mabaila South – Excelle...,OMR 165000,5.0,5.0,NaN,Villa,info@vistaoman.com


In [50]:
import pandas as pd
import numpy as np

# Load the V1.csv file
v1 = pd.read_csv('V1.csv')
print("Original data shape:", v1.shape)
print("Original columns:", list(v1.columns))

# Keep only the columns we want
columns_to_keep = ['title', 'price', 'bedrooms', 'bathrooms', 'city', 'property_type']
existing_keep_columns = [col for col in columns_to_keep if col in v1.columns]
print(f"Columns we can keep: {existing_keep_columns}")

# Drop all the unwanted columns
columns_to_drop = [
    'email', 'phone', 'Unnamed: 8', 'Properties-for-sale-rent...', 
    'Balcony with sea view', 'Fully furnished and move-in ready',
    'Public Parking', 'Running Track', 'Shell & Core', 'Shaded Car Parking',
    'Gymnasium', 'Ladies-Only Pool', 'Large Balcony with Outdoor Space',
    'Guest Room with Attached Washroom'
]

# Only drop columns that actually exist
existing_drop_columns = [col for col in columns_to_drop if col in v1.columns]
if existing_drop_columns:
    v1 = v1.drop(columns=existing_drop_columns)
    print(f"Dropped columns: {existing_drop_columns}")

# Also drop any other columns not in our keep list
all_columns = list(v1.columns)
final_drop_columns = [col for col in all_columns if col not in columns_to_keep]
if final_drop_columns:
    v1 = v1.drop(columns=final_drop_columns)
    print(f"Also dropped: {final_drop_columns}")

print("Remaining columns:", list(v1.columns))
print("New data shape:", v1.shape)

# Clean price column - convert to int
if 'price' in v1.columns:
    # Remove any text and convert to numbers
    v1['price'] = v1['price'].astype(str).str.replace(r'[^\d.]', '', regex=True)
    v1['price'] = pd.to_numeric(v1['price'], errors='coerce')
    
    # Debug: Check what prices we have
    print(f"Sample prices before cleaning: {v1['price'].head(10).tolist()}")
    print(f"Price range: {v1['price'].min()} to {v1['price'].max()}")
    
    # Count how many prices are 5+ figures
    valid_prices_count = (v1['price'] >= 10000).sum()
    print(f"Number of prices with 5+ figures: {valid_prices_count}")
    
    # Only set to 0 if price is less than 10000 AND not missing
    v1['price'] = v1['price'].apply(lambda x: 0 if (pd.notna(x) and x < 10000 and x > 0) else x)
    print(f"Set small prices to 0")
    
    # Fill missing prices with a reasonable default (don't use mean if all are 0)
    missing_count = v1['price'].isna().sum()
    if missing_count > 0:
        v1['price'] = v1['price'].fillna(100000)  # Use fixed reasonable price
        print(f"Filled {missing_count} missing prices with 100000")
    
    # Convert to int
    v1['price'] = v1['price'].astype(int)
    print(f"Sample prices after cleaning: {v1['price'].head(10).tolist()}")
    print(f"Final price range: {v1['price'].min()} to {v1['price'].max()}")

# Clean bedrooms - convert to int
if 'bedrooms' in v1.columns:
    # Extract numbers from text
    v1['bedrooms'] = v1['bedrooms'].astype(str).str.extract(r'(\d+)')[0]
    v1['bedrooms'] = pd.to_numeric(v1['bedrooms'], errors='coerce')
    # Fill missing with mean
    bedrooms_mean = v1['bedrooms'].mean()
    v1['bedrooms'] = v1['bedrooms'].fillna(bedrooms_mean)
    # Convert to int
    v1['bedrooms'] = v1['bedrooms'].astype(int)
    print(f"Bedrooms cleaned - filled missing with mean: {bedrooms_mean:.0f}")

# Clean bathrooms - convert to int
if 'bathrooms' in v1.columns:
    # Extract numbers from text
    v1['bathrooms'] = v1['bathrooms'].astype(str).str.extract(r'(\d+)')[0]
    v1['bathrooms'] = pd.to_numeric(v1['bathrooms'], errors='coerce')
    # Fill missing with mean
    bathrooms_mean = v1['bathrooms'].mean()
    v1['bathrooms'] = v1['bathrooms'].fillna(bathrooms_mean)
    # Convert to int
    v1['bathrooms'] = v1['bathrooms'].astype(int)
    print(f"Bathrooms cleaned - filled missing with mean: {bathrooms_mean:.0f}")

# Clean city column - fill with mode
if 'city' in v1.columns:
    # Fill missing cities with most common city
    city_mode = v1['city'].mode()
    if len(city_mode) > 0:
        city_mode_value = city_mode[0]
        v1['city'] = v1['city'].fillna(city_mode_value)
        print(f"City filled with mode: {city_mode_value}")
    else:
        v1['city'] = v1['city'].fillna('Unknown')
        print("City filled with 'Unknown'")

# Clean property_type column - fill with mode
if 'property_type' in v1.columns:
    # Fill missing property types with most common type
    property_type_mode = v1['property_type'].mode()
    if len(property_type_mode) > 0:
        property_type_mode_value = property_type_mode[0]
        v1['property_type'] = v1['property_type'].fillna(property_type_mode_value)
        print(f"Property type filled with mode: {property_type_mode_value}")
    else:
        v1['property_type'] = v1['property_type'].fillna('Unknown')
        print("Property type filled with 'Unknown'")

# Clean title column - fill missing titles
if 'title' in v1.columns:
    v1['title'] = v1['title'].fillna('Property Listing')
    print("Title cleaned")

# Check for missing values
print("\nMissing values after cleaning:")
print(v1.isnull().sum())

# Show data types
print("\nData types:")
print(v1.dtypes)

# Save cleaned data
v1.to_csv('V1_cleaned.csv', index=False)
print("\nSaved cleaned data to V1_cleaned.csv")
print("Final shape:", v1.shape)

# Show sample data
print("\nFirst 3 rows of cleaned data:")
print(v1.head(3))

# Show some statistics
print("\nData summary:")
if 'price' in v1.columns:
    print(f"Price range: {v1['price'].min():,} to {v1['price'].max():,}")
if 'bedrooms' in v1.columns:
    print(f"Bedrooms range: {v1['bedrooms'].min()} to {v1['bedrooms'].max()}")
if 'bathrooms' in v1.columns:
    print(f"Bathrooms range: {v1['bathrooms'].min()} to {v1['bathrooms'].max()}")
if 'city' in v1.columns:
    print("Top 5 cities:")
    print(v1['city'].value_counts().head())
if 'property_type' in v1.columns:
    print("Property types:")
    print(v1['property_type'].value_counts())

Original data shape: (179, 7)
Original columns: ['title', 'price', 'bedrooms', 'bathrooms', 'city', 'property_type', 'email']
Columns we can keep: ['title', 'price', 'bedrooms', 'bathrooms', 'city', 'property_type']
Dropped columns: ['email']
Remaining columns: ['title', 'price', 'bedrooms', 'bathrooms', 'city', 'property_type']
New data shape: (179, 6)
Sample prices before cleaning: [50000, 20000, 230000, 65000, 165000, 45700, 85000, 110500, 95840, 85000]
Price range: 12320 to 1000000
Number of prices with 5+ figures: 179
Set small prices to 0
Sample prices after cleaning: [50000, 20000, 230000, 65000, 165000, 45700, 85000, 110500, 95840, 85000]
Final price range: 12320 to 1000000
Bedrooms cleaned - filled missing with mean: 3
Bathrooms cleaned - filled missing with mean: 3
City filled with mode: Muscat Hills
Property type filled with mode: Apartment
Title cleaned

Missing values after cleaning:
title            0
price            0
bedrooms         0
bathrooms        0
city          

In [51]:
v1

,title,price,bedrooms,bathrooms,city,property_type
0,1 BR + Maid’s Room Excellent Furnished Apartme...,50000,1,1,Bousher,Apartment
1,1 Desk Office Space for Small Companies in Qurum,20000,2,2,Qurum,Office
2,5 BR + 1 + 1 Rooms Luxury Golf-View Villa with...,230000,5,5,Muscat Hills,Villa
3,1 BR Cozy Freehold Penthouse Apartment – Musca...,65000,1,1,Muscat Hills,Penthouse
4,5 BR Spacious Villa in Mabaila South – Excelle...,165000,5,5,Muscat Hills,Villa
...,...,...,...,...,...,...
174,56 SQ M Smart Offices for Sale,36600,2,2,Muscat Hills,Office
175,3 + 1 BR Courtyard Townhouse in Yiti,238891,1,3,Yiti,Apartment
176,2 BR + 1 Bedroom Brand New Apartment in Shatti...,52500,2,2,Shatti Al Qurum,Apartment
177,77 SQM Brand New Retail Shop at The Gate on No...,12320,2,2,Muscat Hills,Shop


In [59]:
import pandas as pd
import numpy as np

# Load the V1.csv file
v1 = pd.read_csv('V1.csv')
print("Original data shape:", v1.shape)
print("Original columns:", list(v1.columns))

# Keep only the columns we want
columns_to_keep = ['title', 'price', 'bedrooms', 'bathrooms', 'city', 'property_type']
existing_keep_columns = [col for col in columns_to_keep if col in v1.columns]
print(f"Columns we can keep: {existing_keep_columns}")

# Clean price column - convert to int
if 'price' in v1.columns:
    # Remove any text and convert to numbers
    v1['price'] = v1['price'].astype(str).str.replace(r'[^\d.]', '', regex=True)
    v1['price'] = pd.to_numeric(v1['price'], errors='coerce')
    
    # First calculate mean of ONLY valid prices (5+ figures) before setting others to 0
    valid_prices = v1['price'][v1['price'] >= 10000]
    if len(valid_prices) > 0:
        price_mean = valid_prices.mean()
    else:
        price_mean = 50000  # fallback if no valid prices
    
    # Set prices less than 5 figures (less than 10000) to 0
    v1['price'] = v1['price'].apply(lambda x: 0 if x < 10000 else x)
    print(f"Set prices less than 5 figures to 0")
    
    # Fill missing prices with mean of VALID prices only
    v1['price'] = v1['price'].fillna(price_mean)
    # Convert to int
    v1['price'] = v1['price'].astype(int)
    print(f"Price cleaned - filled missing with mean of valid prices: {price_mean:.0f}")

# Clean bedrooms - convert to int
if 'bedrooms' in v1.columns:
    # Extract numbers from text
    v1['bedrooms'] = v1['bedrooms'].astype(str).str.extract(r'(\d+)')[0]
    v1['bedrooms'] = pd.to_numeric(v1['bedrooms'], errors='coerce')
    # Fill missing with mean
    bedrooms_mean = v1['bedrooms'].mean()
    v1['bedrooms'] = v1['bedrooms'].fillna(bedrooms_mean)
    # Convert to int
    v1['bedrooms'] = v1['bedrooms'].astype(int)
    print(f"Bedrooms cleaned - filled missing with mean: {bedrooms_mean:.0f}")

# Clean bathrooms - convert to int
if 'bathrooms' in v1.columns:
    # Extract numbers from text
    v1['bathrooms'] = v1['bathrooms'].astype(str).str.extract(r'(\d+)')[0]
    v1['bathrooms'] = pd.to_numeric(v1['bathrooms'], errors='coerce')
    # Fill missing with mean
    bathrooms_mean = v1['bathrooms'].mean()
    v1['bathrooms'] = v1['bathrooms'].fillna(bathrooms_mean)
    # Convert to int
    v1['bathrooms'] = v1['bathrooms'].astype(int)
    print(f"Bathrooms cleaned - filled missing with mean: {bathrooms_mean:.0f}")

#city cleaning
# Example list of known Omani cities/areas to match in titles
known_cities = ['Ruwi', 'Al Hail', 'Boushar', ' Bousher' , 'Mabela','Muscat', 'Azaiba', 'Seeb', 'Mawaleh', 'Qurum', 'Salalah', 'Madinat Sultan Qaboos', 'Ghubra', 'Muscat Hills']

def extract_city_from_title(title):
    if pd.isnull(title):
        return None
    for city in known_cities:
        if city.lower() in title.lower():
            return city
    return None

# Apply extraction if 'title' column exists
if 'title' in v1.columns:
    v1['city_extracted'] = v1['title'].apply(extract_city_from_title)

# Use 'city_extracted' to fill missing city values
if 'city' in v1.columns:
    v1['city'] = v1['city'].fillna(v1['city_extracted'])

# Then fall back to filling remaining missing values with mode
city_mode = v1['city'].mode()
if len(city_mode) > 0:
    city_mode_value = city_mode[0]
    v1['city'] = v1['city'].fillna(city_mode_value)
    print(f"City filled using extracted value or mode: {city_mode_value}")
else:
    v1['city'] = v1['city'].fillna('Unknown')
    print("City filled with 'Unknown'")


# Clean property_type column - fill with mode
if 'property_type' in v1.columns:
    # Fill missing property types with most common type
    property_type_mode = v1['property_type'].mode()
    if len(property_type_mode) > 0:
        property_type_mode_value = property_type_mode[0]
        v1['property_type'] = v1['property_type'].fillna(property_type_mode_value)
        print(f"Property type filled with mode: {property_type_mode_value}")
    else:
        v1['property_type'] = v1['property_type'].fillna('Unknown')
        print("Property type filled with 'Unknown'")

# Clean title column - fill missing titles
if 'title' in v1.columns:
    v1['title'] = v1['title'].fillna('Property Listing')
    print("Title cleaned")

# Check for missing values
print("\nMissing values after cleaning:")
print(v1.isnull().sum())

# Show data types
print("\nData types:")
print(v1.dtypes)

# Save cleaned data
v1.to_csv('V1_cleaned.csv', index=False)
print("\nSaved cleaned data to V1_cleaned.csv")
print("Final shape:", v1.shape)

# Show sample data
print("\nFirst 3 rows of cleaned data:")
print(v1.head(3))

# Show some statistics
print("\nData summary:")
if 'price' in v1.columns:
    print(f"Price range: {v1['price'].min():,} to {v1['price'].max():,}")
if 'bedrooms' in v1.columns:
    print(f"Bedrooms range: {v1['bedrooms'].min()} to {v1['bedrooms'].max()}")
if 'bathrooms' in v1.columns:
    print(f"Bathrooms range: {v1['bathrooms'].min()} to {v1['bathrooms'].max()}")
if 'city' in v1.columns:
    print("Top 5 cities:")
    print(v1['city'].value_counts().head())
if 'property_type' in v1.columns:
    print("Property types:")
    print(v1['property_type'].value_counts())

Original data shape: (179, 7)
Original columns: ['title', 'price', 'bedrooms', 'bathrooms', 'city', 'property_type', 'email']
Columns we can keep: ['title', 'price', 'bedrooms', 'bathrooms', 'city', 'property_type']
Set prices less than 5 figures to 0
Price cleaned - filled missing with mean of valid prices: 160080
Bedrooms cleaned - filled missing with mean: 3
Bathrooms cleaned - filled missing with mean: 3
City filled using extracted value or mode: Muscat
Property type filled with mode: Apartment
Title cleaned

Missing values after cleaning:
title              0
price              0
bedrooms           0
bathrooms          0
city               0
property_type      0
email              0
city_extracted    90
dtype: int64

Data types:
title             object
price              int64
bedrooms           int64
bathrooms          int64
city              object
property_type     object
email             object
city_extracted    object
dtype: object

Saved cleaned data to V1_cleaned.csv
Fina

In [58]:
v1

,title,price,bedrooms,bathrooms,city,property_type,email,city_extracted
0,1 BR + Maid’s Room Excellent Furnished Apartme...,50000,1,1,Bousher,Apartment,info@vistaoman.com,None
1,1 Desk Office Space for Small Companies in Qurum,20000,2,2,Qurum,Office,info@vistaoman.com,Qurum
2,5 BR + 1 + 1 Rooms Luxury Golf-View Villa with...,230000,5,5,Muscat Hills,Villa,info@vistaoman.com,None
3,1 BR Cozy Freehold Penthouse Apartment – Musca...,65000,1,1,Qurum,Penthouse,info@vistaoman.com,None
4,5 BR Spacious Villa in Mabaila South – Excelle...,165000,5,5,Qurum,Villa,info@vistaoman.com,None
...,...,...,...,...,...,...,...,...
174,56 SQ M Smart Offices for Sale,36600,2,2,Qurum,Office,info@vistaoman.com,None
175,3 + 1 BR Courtyard Townhouse in Yiti,238891,1,3,Yiti,Apartment,info@vistaoman.com,None
176,2 BR + 1 Bedroom Brand New Apartment in Shatti...,52500,2,2,Shatti Al Qurum,Apartment,info@vistaoman.com,Qurum
177,77 SQM Brand New Retail Shop at The Gate on No...,12320,2,2,Qurum,Shop,info@vistaoman.com,None


# merging both datas together 

In [54]:
import pandas as pd

# Load the two cleaned files
tibiaan = pd.read_csv('cleaned_tibiaan.csv')
v1 = pd.read_csv('V1_cleaned.csv')

print("Tibiaan shape:", tibiaan.shape)
print("V1 shape:", v1.shape)

# Check what columns each file has
print("Tibiaan columns:", list(tibiaan.columns))
print("V1 columns:", list(v1.columns))

# Make column names match - change tibiaan column names to lowercase
tibiaan.columns = tibiaan.columns.str.lower()
print("Tibiaan columns after lowercase:", list(tibiaan.columns))

# Now both should have same column names
print("Checking if columns match now...")
tibiaan_cols = set(tibiaan.columns)
v1_cols = set(v1.columns)
common_cols = list(tibiaan_cols & v1_cols)
print("Common columns:", common_cols)

# Keep only the common columns in both datasets
tibiaan_keep = tibiaan[common_cols].copy()
v1_keep = v1[common_cols].copy()

print("After keeping common columns:")
print("Tibiaan shape:", tibiaan_keep.shape)
print("V1 shape:", v1_keep.shape)

# Add a column to know which dataset each row came from
tibiaan_keep['source'] = 'tibiaan'
v1_keep['source'] = 'v1'

# Combine both datasets
merged = pd.concat([tibiaan_keep, v1_keep])

print("Merged data shape:", merged.shape)
print("Merged columns:", list(merged.columns))

# Check how many from each source
print("Data from each source:")
print(merged['source'].value_counts())


# Save the merged data
merged.to_csv('merged_final.csv', index=False)
print("Saved merged data to merged_final.csv")

# Show first few rows
print("First 5 rows:")
print(merged.head())

Tibiaan shape: (379, 7)
V1 shape: (179, 6)
Tibiaan columns: ['Title', 'Location', 'Price', 'Size', 'Bedrooms', 'Bathrooms', 'Property_Type']
V1 columns: ['title', 'price', 'bedrooms', 'bathrooms', 'city', 'property_type']
Tibiaan columns after lowercase: ['title', 'location', 'price', 'size', 'bedrooms', 'bathrooms', 'property_type']
Checking if columns match now...
Common columns: ['bedrooms', 'bathrooms', 'property_type', 'title', 'price']
After keeping common columns:
Tibiaan shape: (379, 5)
V1 shape: (179, 5)
Merged data shape: (558, 6)
Merged columns: ['bedrooms', 'bathrooms', 'property_type', 'title', 'price', 'source']
Data from each source:
source
tibiaan    379
v1         179
Name: count, dtype: int64
Saved merged data to merged_final.csv
First 5 rows:
   bedrooms  bathrooms property_type  \
0         2          3         Villa   
1         8         11         Villa   
2         2          3     Apartment   
3         2          3     Apartment   
4         5          5      

In [55]:
merged

,bedrooms,bathrooms,property_type,title,price,source
0,2,3,Villa,Furnished Villa for Sale in Al Maabela – Resor...,80000,tibiaan
1,8,11,Villa,Luxury furnished villa for sale with 8 bedroom...,415000,tibiaan
2,2,3,Apartment,Newly built Flat for Sale in Salalah,55000,tibiaan
3,2,3,Apartment,Flat for sale in Al Qurum,45000,tibiaan
4,5,5,Land,Land for slae in Al Qurum,525000,tibiaan
...,...,...,...,...,...,...
174,2,2,Office,56 SQ M Smart Offices for Sale,36600,v1
175,1,3,Apartment,3 + 1 BR Courtyard Townhouse in Yiti,238891,v1
176,2,2,Apartment,2 BR + 1 Bedroom Brand New Apartment in Shatti...,52500,v1
177,2,2,Shop,77 SQM Brand New Retail Shop at The Gate on No...,12320,v1
